In [1]:
import json
import numpy as np
import pandas as pd
import lxml
import re
import os
from datetime import datetime

In [3]:
df = pd.read_json('data/execution/script_history.json')

In [20]:
try:
    os.remove('data/execution/script_history_fixed.json')
except:
    print('File does not exist')

with open('data/execution/script_history.json', 'r') as fr:
    # reading line by line
    lines = fr.readlines()
    last_line = len(lines)
    print(len(lines))
    
    for line in lines:
        line = re.sub("\u0003", "", line)     
        #line = re.sub("Click Dynamics 365 — custom\nProvides access to the full suite of capabilities, including administration and settings\nMicrosoft Dynamics 365\n11/23/2022\nWEB","Click Dynamics 365",line)
        with open('data/execution/script_history_fixed.json', 'a') as fw:
            fw.write(line) 

590998


In [21]:
f = open('data/execution/script_history_fixed.json')
data = json.loads(f.read())

In [60]:
try:
    os.remove('data/execution/check_structure.csv')
except:
    print('File does not exist')
with open('data/execution/check_structure.csv', 'a') as fw:
    line = 'records_no,topProjectName,name,sessionId,id,scriptId,scriptName,state,status,label,startTime,endTime,duration,lineNum,StepLogs_id\n'
    fw.write(line)
    i = 0
    for level1 in data:
        for key, value in level1.items():
            #if i == 1000: break
            i = i + 1
            try:
                executionLogs_str = json.dumps(level1['executionLogs'])
                executionLogs_json = json.loads(executionLogs_str)
                for level2 in executionLogs_json:
                    for key2, value2 in level2.items():
                        #try:
                        executionStepLogs_str = json.dumps(level2['executionStepLogs'])
                        executionStepLogs_json = json.loads(executionStepLogs_str)
                        if key2 == 'executionStepLogs':
                            records_no_str = str(i)
                            topProjectName = str(level1['topProjectName'])
                            name_str = str(level1['name'])
                            # Remove any commas from name 
                            check = (',' in name_str)
                            if check == True:
                                name_str = re.sub(',', ';', name_str)                            
                            sessionId_str = str(level2['sessionId'])
                            id_str = str(level2['id'])
                            scriptId_str = str(level2['scriptId'])
                            scriptName_str = str(level2['scriptName'])
                            # Remove any commas from scriptName 
                            check = (',' in scriptName_str)
                            if check == True:
                                scriptName_str = re.sub(',', ';', scriptName_str)                             
                            state_str = str(level2['state'])
                            status_str = str(level2['status'])
                            lastIssues_str = str(level2['lastIssues'])
                            # Remove any commas from lastIssues_str
                            check = (',' in lastIssues_str)
                            if check == True:
                                lastIssues_str = re.sub(',', ';', lastIssues_str)


                            for level3 in executionStepLogs_json:
                                #for key3, value3 in level3.items():
                                status_str = str(level3['status'])
                                label_str = str(level3['label'])
                                
                                # These lines replace long text test step labels with short text (that does not wrap)
                                check = ('Click Dynamics 365' in label_str)
                                if check == True:
                                    label_str = 'Click Dynamics 365'
                                check = ('Click Sales Hub' in label_str)
                                if check == True:
                                    label_str = 'Click Sales Hub'
                                check = ('Type value function eaExecuteVariableValue()' in label_str)
                                if check == True:
                                    label_str = 'Type value function eaExecuteVariableValue()'
                                check = ('Select from function eaExecuteVariableValue()' in label_str)
                                if check == True:
                                    label_str = 'Select from function eaExecuteVariableValue()'                                    
                                check = ('Select lookup values function eaExecuteVariableValue()' in label_str)
                                if check == True:
                                    label_str = 'Select lookup values function eaExecuteVariableValue()'
                                # Remove any commas from label_str 
                                check = (',' in label_str)
                                if check == True:
                                    label_str = re.sub(',', ';', label_str)
                                # Now replace all newlines that remain in label_str
                                check = ('\n' in label_str)
                                if check == True:
                                    label_str = re.sub('\n', '/nl/', label_str)
                                # Done with replacing text

                                startTime_str = str(level3['startTime'])
                                endTime_str = str(level3['endTime'])
                                duration_str = str(level3['duration'])
                                lineNum_str = str(level3['lineNum'])
                                StepLogs_id_str = str(level3['id'])

                                line = records_no_str+','+topProjectName+','+name_str+','+sessionId_str+','+id_str+','+scriptId_str+','+scriptName_str+','+state_str+','+status_str+','+label_str+','+startTime_str+','+endTime_str+','+duration_str+','+lineNum_str+','+StepLogs_id_str+'\n'
                                fw.write(line)
                            # except:
                            #     print('Exception on Level2 for Level1 item '+str(i))
            except:
                print('Exception for Level1 item '+str(i))


Exception for Level1 item 190
Exception for Level1 item 191
Exception for Level1 item 192
Exception for Level1 item 193
Exception for Level1 item 194
Exception for Level1 item 195
Exception for Level1 item 196
Exception for Level1 item 197
Exception for Level1 item 198
Exception for Level1 item 199
Exception for Level1 item 200
Exception for Level1 item 201
Exception for Level1 item 202
Exception for Level1 item 203
Exception for Level1 item 204
Exception for Level1 item 205
Exception for Level1 item 206
Exception for Level1 item 207
Exception for Level1 item 208
Exception for Level1 item 209
Exception for Level1 item 231
Exception for Level1 item 232
Exception for Level1 item 233
Exception for Level1 item 234
Exception for Level1 item 235
Exception for Level1 item 236
Exception for Level1 item 237
Exception for Level1 item 238
Exception for Level1 item 239
Exception for Level1 item 240
Exception for Level1 item 241
Exception for Level1 item 242
Exception for Level1 item 243
Exception 

In [61]:
df = pd.read_csv('data/execution/check_structure.csv')

In [62]:
#df.drop_duplicates(keep='first',inplace=True)
df.rename(columns={'label':'StepLabel'}, inplace=True)

In [63]:
mask = (df['startTime'] == 'None') 
df['startTime'].mask(mask,'2022-01-01T00:00:00.000Z',inplace=True)
df['startTime'] = pd.to_datetime(df['startTime'])

In [64]:
mask = (df['endTime'] == 'None') 
df['endTime'].mask(mask,'2022-01-01T00:00:00.000Z',inplace=True)
df['endTime'] = pd.to_datetime(df['endTime'])

In [65]:
df['duration_ms'] = df['endTime'] - df['startTime']

In [69]:
mask = (df['duration'] == 'None') 
df['duration'].mask(mask,'0',inplace=True)
df['duration'] = pd.to_numeric(df['duration'])

In [70]:
df.head(50)

,records_no,topProjectName,name,sessionId,id,scriptId,scriptName,state,status,StepLabel,startTime,endTime,duration,lineNum,StepLogs_id,duration_ms
0,1,D365FO CCBA PLAN,Test,cEbKnIVBIR,1,1,Test,TERMINATED,ERROR,Start script,2022-11-10 08:12:33.833000+00:00,2022-11-10 08:12:33.874000+00:00,41,1,1,0 days 00:00:00.041000
1,2,D365FO CCBA PLAN,Test,cEbKnIVBIR,1,1,Test,TERMINATED,ERROR,Start script,2022-11-10 08:12:33.833000+00:00,2022-11-10 08:12:33.874000+00:00,41,1,1,0 days 00:00:00.041000
2,3,D365FO CCBA PLAN,Test,cEbKnIVBIR,1,1,Test,TERMINATED,ERROR,Start script,2022-11-10 08:12:33.833000+00:00,2022-11-10 08:12:33.874000+00:00,41,1,1,0 days 00:00:00.041000
3,4,D365FO CCBA PLAN,Test,cEbKnIVBIR,1,1,Test,TERMINATED,ERROR,Start script,2022-11-10 08:12:33.833000+00:00,2022-11-10 08:12:33.874000+00:00,41,1,1,0 days 00:00:00.041000
4,5,D365FO CCBA PLAN,Test,cEbKnIVBIR,1,1,Test,TERMINATED,ERROR,Start script,2022-11-10 08:12:33.833000+00:00,2022-11-10 08:12:33.874000+00:00,41,1,1,0 days 00:00:00.041000
5,6,D365FO CCBA PLAN,Test,cEbKnIVBIR,1,1,Test,TERMINATED,ERROR,Start script,2022-11-10 08:12:33.833000+00:00,2022-11-10 08:12:33.874000+00:00,41,1,1,0 days 00:00:00.041000
6,7,D365FO CCBA PLAN,Test,cEbKnIVBIR,1,1,Test,TERMINATED,ERROR,Start script,2022-11-10 08:12:33.833000+00:00,2022-11-10 08:12:33.874000+00:00,41,1,1,0 days 00:00:00.041000
7,8,D365FO CCBA PLAN,Test,cEbKnIVBIR,1,1,Test,TERMINATED,ERROR,Start script,2022-11-10 08:12:33.833000+00:00,2022-11-10 08:12:33.874000+00:00,41,1,1,0 days 00:00:00.041000
8,9,D365FO CCBA PLAN,Test,cEbKnIVBIR,1,1,Test,TERMINATED,ERROR,Start script,2022-11-10 08:12:33.833000+00:00,2022-11-10 08:12:33.874000+00:00,41,1,1,0 days 00:00:00.041000
9,10,D365FO CCBA PLAN,Test,cEbKnIVBIR,1,1,Test,TERMINATED,ERROR,Start script,2022-11-10 08:12:33.833000+00:00,2022-11-10 08:12:33.874000+00:00,41,1,1,0 days 00:00:00.041000


In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1095293 entries, 0 to 1095292
Data columns (total 16 columns):
 #   Column          Non-Null Count    Dtype              
---  ------          --------------    -----              
 0   records_no      1095293 non-null  int64              
 1   topProjectName  1095293 non-null  object             
 2   name            1095293 non-null  object             
 3   sessionId       1095293 non-null  object             
 4   id              1095293 non-null  int64              
 5   scriptId        1095293 non-null  int64              
 6   scriptName      1095293 non-null  object             
 7   state           1095293 non-null  object             
 8   status          1095293 non-null  object             
 9   StepLabel       1095293 non-null  object             
 10  startTime       1095293 non-null  datetime64[ns, UTC]
 11  endTime         1095293 non-null  datetime64[ns, UTC]
 12  duration        1095293 non-null  int64              
 1